In [1]:
import pandas as pd
import json

In [2]:
charity = pd.read_csv('charity.csv')

In [3]:
print(charity.shape)
charity.head()

(357, 11)


,Rental flat Y/N,Postal Code,Address,Total units,1-room (units),2-room (units),3-room (units),Charities serving block,Demographic,Food needed,Frequency needed
0,NaN,050005,Blk 5 Banda Street,430.0,430,0,0,NaN,NaN,NaN,NaN
1,NaN,080333,Blk 333 Kreta Ayer Road,126.0,0,126,0,NaN,NaN,NaN,NaN
2,NaN,090003,Blk 3 Telok Blangah Crescent,264.0,264,0,0,THK SAC @ Telok Blangah Crescent / Apex Club o...,NaN,NaN,NaN
3,NaN,090004,Blk 4 Telok Blangah Crescent,264.0,264,0,0,THK SAC @ Telok Blangah Crescent @ Blk 3,NaN,NaN,NaN
4,NaN,090019,Blk 19 Telok Blangah Crescent,364.0,364,0,0,NaN,NaN,NaN,NaN


In [4]:
charity.columns

Index(['Rental flat Y/N', 'Postal Code', 'Address', 'Total units',
       '1-room (units)', '2-room (units)', '3-room (units)',
       'Charities serving block', 'Demographic', 'Food needed',
       'Frequency needed'],
      dtype='object')

In [5]:
buildings = pd.read_json('buildings.json')
print(buildings.shape)
buildings.head()

(141791, 11)


,ADDRESS,BLK_NO,BUILDING,LATITUDE,LONGITUDE,LONGTITUDE,POSTAL,ROAD_NAME,SEARCHVAL,X,Y
0,1 STRAITS BOULEVARD SINGAPORE CHINESE CULTURAL...,1,SINGAPORE CHINESE CULTURAL CENTRE,1.275829,103.849576,103.849576,018906,STRAITS BOULEVARD,SINGAPORE CHINESE CULTURAL CENTRE,29809.365407,28700.236127
1,11A STRAITS BOULEVARD TEMPORARY SITE OFFICE SI...,11A,TEMPORARY SITE OFFICE,1.274950,103.851665,103.851665,018907,STRAITS BOULEVARD,TEMPORARY SITE OFFICE,30041.838898,28602.987244
2,5A MARINA GARDENS DRIVE SINGAPORE 018910,5A,NIL,1.279587,103.868956,103.868956,018910,MARINA GARDENS DRIVE,5A MARINA GARDENS DRIVE SINGAPORE 018910,31966.120787,29115.753373
3,100A CENTRAL BOULEVARD TEMPORARY SITE OFFICE S...,100A,TEMPORARY SITE OFFICE,1.273682,103.860075,103.860075,018915,CENTRAL BOULEVARD,TEMPORARY SITE OFFICE,30977.832467,28462.795296
4,2 CENTRAL BOULEVARD CENTRAL BOULEVARD TOWERS S...,2,CENTRAL BOULEVARD TOWERS,1.279785,103.851529,103.851529,018916,CENTRAL BOULEVARD,CENTRAL BOULEVARD TOWERS,30026.620277,29137.649384


In [6]:
merged = pd.merge(charity, buildings[['POSTAL', 'LATITUDE', 'LONGITUDE']].drop_duplicates(subset=['POSTAL']),
                 left_on='Postal Code', right_on='POSTAL',
                 how='left')

In [7]:
merged.head()

,Rental flat Y/N,Postal Code,Address,Total units,1-room (units),2-room (units),3-room (units),Charities serving block,Demographic,Food needed,Frequency needed,POSTAL,LATITUDE,LONGITUDE
0,NaN,050005,Blk 5 Banda Street,430.0,430,0,0,NaN,NaN,NaN,NaN,050005,1.281197,103.843437
1,NaN,080333,Blk 333 Kreta Ayer Road,126.0,0,126,0,NaN,NaN,NaN,NaN,080333,1.280748,103.842463
2,NaN,090003,Blk 3 Telok Blangah Crescent,264.0,264,0,0,THK SAC @ Telok Blangah Crescent / Apex Club o...,NaN,NaN,NaN,090003,1.278668,103.817527
3,NaN,090004,Blk 4 Telok Blangah Crescent,264.0,264,0,0,THK SAC @ Telok Blangah Crescent @ Blk 3,NaN,NaN,NaN,090004,1.278760,103.818443
4,NaN,090019,Blk 19 Telok Blangah Crescent,364.0,364,0,0,NaN,NaN,NaN,NaN,090019,1.277463,103.822653


# map

In [8]:
import folium
from folium.plugins import MarkerCluster, HeatMap, HeatMapWithTime

## demand map

In [21]:
mymap = folium.Map(location=[1.3521, 103.8198],
                    zoom_start = 11) # Uses lat then lon. The bigger the zoom number, the closer in you get

In [22]:
def add_markers(df, mymap):
    """
    Given the json response of carparks and the base map,
    label each carpark with a marker on the map 
    """
    
    marker_cluster = MarkerCluster().add_to(mymap)
    
    for ind, row in df.iterrows():
        if row['Charities serving block']:
            folium.Marker([row['LATITUDE'], row['LONGITUDE']],
                          tooltip=row['Charities serving block']
                         ).add_to(marker_cluster)
        else:
            folium.Marker([row['LATITUDE'], row['LONGITUDE']],
                          tooltip="None"
                         ).add_to(marker_cluster)
    
    return mymap

In [23]:
mymap = folium.Map(location=[1.3521, 103.8198],
                    zoom_start = 11) # Uses lat then lon. The bigger the zoom number, the closer in you get

mymap = add_markers(merged.dropna(subset=['LATITUDE', 'LONGITUDE']), mymap)

mymap # Calls the map to display

In [24]:
mymap.save('demand.html')

## supply map

In [25]:
mymap = folium.Map(location=[1.3521, 103.8198],
                    zoom_start = 11) # Uses lat then lon. The bigger the zoom number, the closer in you get

In [26]:
def add_markers(df, mymap):
    """
    Given the json response of carparks and the base map,
    label each carpark with a marker on the map 
    """
    
    marker_cluster = MarkerCluster().add_to(mymap)
    
    for ind, row in df.dropna(subset=['Charities serving block']).iterrows():
        folium.Marker([row['LATITUDE'], row['LONGITUDE']],
                      tooltip=row['Charities serving block']
                     ).add_to(marker_cluster)
    
    return mymap

In [27]:
mymap = folium.Map(location=[1.3521, 103.8198],
                    zoom_start = 11) # Uses lat then lon. The bigger the zoom number, the closer in you get

mymap = add_markers(merged, mymap)

mymap # Calls the map to display

In [28]:
mymap.save('supply.html')